# **Reduction of Quantum Circuits using CLUE**$\def\bx{\mathbf{x}}\def\quant#1{\left|#1\right\rangle}\def\cc{\mathbb{C}}$

In [1]:
import sys; sys.path.insert(0, "../..") # clue is here
from clue.qiskit import *
from numpy import array, matmul, cdouble, sqrt, arccos, cos, sin
from numpy.random import rand

## 1. Description of methodology

In this notebook we show how we can use CLUE to reduce a Quantum circuit described in `qasm`. Without really understanding what is going on under the hood, we know that, given a quantum circuit there is a unitary matrix associated that represents the quantum computations that are done in the circuit.

In my current intuition, if the circuit hsa $n$ q-bits, then there is a complex $2^n\times 2^n$ matrix $A$ that act like a "Markov matrix" over a valid superposition of states. Namely, if we have the state $\bx = (x_0,\ldots,x_{2^n-1})$ where the $x_i$ entry represent the part of the superposition corresponding with the state $\quant{i}$, then we obtain the state $A\bx^T$ after applying the quantum circuit.

We would like to get a reduction (lumping) $L \in \cc^{m\times 2^n}$ for this matrix $A$ that helps to reduce the size or simplify the understanding of the circuit. For doing so we do the following:

1. Read a `qasm` file as taken from the benchmarks in this website: [Benchmarks](https://www.cda.cit.tum.de/mqtbench/)
2. We compute the unitary matrix $A$ for the circuit:
   * We need to remove all measures from the `qasm` file.
   * It seems the software `qiskit` does simulations over the circuit in order to compute this unitary matrix.
3. We use this matrix to set up a dynamical system of the form $\bx_{k+1}^T = A\bx_k^T$ (this system is not actually happening, but may be interesting to remember).
4. We run CLUE for this system for a given observable (currently we take one of the variables) $\longrightarrow$ we get a matrix $L$.
   * We run *approximate CLUE* since the unitary matrix has double precision complex numbers as inputs.
   * We have adapted the code of CLUE to *run over the complex numbers* too.
   * In some cases, when we get a reduction it is an exact lumping (at least in the QFT models that has been checked by hand).
   * The matrix $L$ is orthonormal, meaning that the reduced system is defined again by a unitary matrix.

## 2. Some examples 

Let us show in the code how to compute these reductions using CLUE for the QFT algorithm with 3 q-bits. 

**Remark**: the `qasm` files are included in the folder `circuits`.

In [2]:
system = DS_QuantumCircuit("circuits/qft_indep_qiskit_3.qasm"); system

circuit-117 [DS_QuantumCircuit -- 8 -- SparsePolynomial]

### The data in the system

We can show the equations of the system:

In [3]:
for (i,equ) in enumerate(system.equations): 
    print(i, " -> ", equ)

0  ->  (0.353553390593274 + 0.0j)*Q_000 + (0.353553390593274 - 4.32978028117747e-17j)*Q_001 + (0.353553390593274 - 4.32978028117747e-17j)*Q_010 + (0.353553390593274 - 8.65956056235493e-17j)*Q_011 + (0.353553390593274 - 4.32978028117747e-17j)*Q_100 + (0.353553390593274 - 8.65956056235493e-17j)*Q_101 + (0.353553390593274 - 8.65956056235493e-17j)*Q_110 + (0.353553390593274 - 1.29893408435324e-16j)*Q_111
1  ->  (0.353553390593274 + 0.0j)*Q_000 + (0.25 + 0.25j)*Q_001 + (6.4946704217662e-17 + 0.353553390593274j)*Q_010 + (-0.25 + 0.25j)*Q_011 + (-0.353553390593274 + 4.32978028117747e-17j)*Q_100 + (-0.25 - 0.25j)*Q_101 + (-1.08244507029437e-16 - 0.353553390593274j)*Q_110 + (0.25 - 0.25j)*Q_111
2  ->  (0.353553390593274 + 0.0j)*Q_000 + (6.4946704217662e-17 + 0.353553390593274j)*Q_001 + (-0.353553390593274 + 4.32978028117747e-17j)*Q_010 + (-1.08244507029437e-16 - 0.353553390593274j)*Q_011 + (0.353553390593274 - 4.32978028117747e-17j)*Q_100 + (1.08244507029437e-16 + 0.353553390593274j)*Q_101 + (-

Or, since the system is linear, we can show the unitary matrix from the system:

In [4]:
A = system.construct_matrices("polynomial")[0].to_numpy(dtype=cdouble)
for (i,row) in enumerate(A.round(5)):
    print(i, " -> ", row)

0  ->  [0.35355+0.j 0.35355+0.j 0.35355+0.j 0.35355+0.j 0.35355+0.j 0.35355+0.j
 0.35355+0.j 0.35355+0.j]
1  ->  [ 0.35355-0.j       0.25   +0.25j     0.     +0.35355j -0.25   +0.25j
 -0.35355+0.j      -0.25   -0.25j    -0.     -0.35355j  0.25   -0.25j   ]
2  ->  [ 0.35355-0.j       0.     +0.35355j -0.35355+0.j      -0.     -0.35355j
  0.35355-0.j       0.     +0.35355j -0.35355+0.j      -0.     -0.35355j]
3  ->  [ 0.35355-0.j      -0.25   +0.25j    -0.     -0.35355j  0.25   +0.25j
 -0.35355+0.j       0.25   -0.25j     0.     +0.35355j -0.25   -0.25j   ]
4  ->  [ 0.35355-0.j -0.35355+0.j  0.35355-0.j -0.35355+0.j  0.35355-0.j
 -0.35355+0.j  0.35355-0.j -0.35355+0.j]
5  ->  [ 0.35355-0.j      -0.25   -0.25j     0.     +0.35355j  0.25   -0.25j
 -0.35355+0.j       0.25   +0.25j    -0.     -0.35355j -0.25   +0.25j   ]
6  ->  [ 0.35355-0.j      -0.     -0.35355j -0.35355+0.j       0.     +0.35355j
  0.35355-0.j      -0.     -0.35355j -0.35355+0.j       0.     +0.35355j]
7  ->  [ 0.35355-0.

### Computing the lumping

Let say we want to compute the lumping with respect to the state $\quant{100} = \quant{4}$. We can do that with the following code:

In [5]:
lumped = system.lumping([system.variables[4]])

New variables:
y0 = Q_100
y1 = (0.377964473009227 + 0.0j)*Q_000 + (-0.377964473009227 - 4.62872982062167e-17j)*Q_001 + (0.377964473009227 + 4.62872982062166e-17j)*Q_010 + (-0.377964473009227 - 9.25745964124333e-17j)*Q_011 + (-0.377964473009227 - 9.25745964124333e-17j)*Q_101 + (0.377964473009227 + 9.25745964124332e-17j)*Q_110 + (-0.377964473009227 - 1.3886189461865e-16j)*Q_111
New initial conditions:
Lumped system:
y0' = (0.935414346693485 + 7.8702812005206e-33j)*y1 + (0.353553390593274 - 4.32978028117747e-17j)*y0
y1' = (-0.353553390593274 - 7.32847151738175e-17j)*y1 + (0.935414346693485 + 6.54601248888394e-17j)*y0


We can get the lumping matrix (as read from the previous output):

In [6]:
L = lumped.lumping_matrix.to_numpy(dtype=cdouble); L

array([[ 0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         1.        +0.00000000e+00j,  0.        +0.00000000e+00j,
         0.        +0.00000000e+00j,  0.        +0.00000000e+00j],
       [ 0.37796447+0.00000000e+00j, -0.37796447-4.62872982e-17j,
         0.37796447+4.62872982e-17j, -0.37796447-9.25745964e-17j,
         0.        +0.00000000e+00j, -0.37796447-9.25745964e-17j,
         0.37796447+9.25745964e-17j, -0.37796447-1.38861895e-16j]])

And we can check that the matrix for the new system is unitary:

In [7]:
A = lumped.construct_matrices("polynomial")[0].to_numpy(dtype=cdouble)
matmul(A, A.transpose().conjugate()).round(15)

array([[1.+0.j, 0.+0.j],
       [0.-0.j, 1.+0.j]])

## 3. Compiling some results

Here we present a table with some results after running the lumping algorithm for several observables:

In [8]:
import pandas as pd

In [11]:
data = pd.read_csv("./compilation.csv", delimiter=";"); data

,circuit,observable,size,reduced,ratio,time
0,qft_indep_qiskit_7,'Q_0000000',128,2,0.015625,2.117150
1,qft_indep_qiskit_7,'Q_0000001',128,128,1.000000,196.215425
2,qft_indep_qiskit_7,'Q_0000010',128,128,1.000000,142.706798
3,qft_indep_qiskit_7,'Q_0000011',128,128,1.000000,146.102181
4,qft_indep_qiskit_7,'Q_0000100',128,128,1.000000,180.548844
...,...,...,...,...,...,...
760,qft_indep_qiskit_6,'Q_111011',64,64,1.000000,15.257647
761,qft_indep_qiskit_6,'Q_111100',64,64,1.000000,15.048586
762,qft_indep_qiskit_6,'Q_111101',64,64,1.000000,15.284256
763,qft_indep_qiskit_6,'Q_111110',64,64,1.000000,15.581070


Let us try to filter those cases that have an actual reduction:

In [13]:
data[data["ratio"] < 1].sort_values("size")

,circuit,observable,size,reduced,ratio,time
485,qft_indep_qiskit_2,"'Q_01', 'Q_11'",4,3,0.750000,0.002920
482,qft_indep_qiskit_2,"'Q_00', 'Q_10'",4,3,0.750000,0.003687
479,qft_indep_qiskit_2,'Q_10',4,2,0.500000,0.003160
477,qft_indep_qiskit_2,'Q_00',4,2,0.500000,0.004954
226,qft_indep_qiskit_3,'Q_100',8,2,0.250000,0.007432
...,...,...,...,...,...,...
568,grover-v-chain_indep_qiskit_9,'Q_001001011',512,6,0.011719,71.340642
569,grover-v-chain_indep_qiskit_9,'Q_001001100',512,6,0.011719,74.741405
557,grover-v-chain_indep_qiskit_9,'Q_001000000',512,6,0.011719,64.956130
525,grover-v-chain_indep_qiskit_9,'Q_000100000',512,6,0.011719,64.181567


## 4. Some comments:

* If we have $n$ qbits, then our matrices have size $2^{n}\times 2^n$. Hence, we have a total of $2^{2n}$ elements in the matrix. Each element is a double precision complex number, hence it needs at least $16=2^{4}$ bytes per element of the matrix. Hence, in order to build the unitary matrix (which goes through numpy and its dense representation) we will require at least: $2^{2(n+2)}$ bytes.
* In the particular case of 11 qbits: $2^{26}\text{B} = 64\text{MB}$.
* With 1 GB of memory we could represent the unitary matrix of at most 13 qbits.

## 5. Grove's algorithm

Grove's algorithm performs an unstructure search in a quantum state. Intuitively, the Grover gate depends on an oracle gate (i.e., another circuit) that maps each of the base states to itself when the state satisfies the search condition and to its additive inverse if the search criteria does not fit. 

In the [benchmark](https://www.cda.cit.tum.de/mqtbench/benchmark_description) is explained that the Grove circuit is built with an Oracle created from a [Toffoli gate](https://en.wikipedia.org/wiki/Toffoli_gate). How does this work? We do not completely understand.

In [2]:
system = DS_QuantumCircuit("circuits/grover-v-chain_indep_qiskit_7.qasm"); system

circuit-117 [DS_QuantumCircuit -- 128 -- SparsePolynomial]

## 6. More experiments?

This place is for further experimentation on the fly

In [2]:
system = DS_QuantumCircuit.from_qasm_file("./circuits/ghz_indep_qiskit_6.qasm")
obs = [SparsePolynomial.from_string(f"{1/sqrt(2)}*{system.variables[0]} + {1/sqrt(2)}*{system.variables[-1]}", system.variables, system.field)]
goal = SparsePolynomial.from_string(f"{system.variables[0]}", system.variables, system.field)

In [3]:
obs

[(0.707106781186547 + 0.0j)*Q_000000 + (0.707106781186547 + 0.0j)*Q_111111]

In [4]:
lumped = system.lumping(obs, print_reduction=False, print_system=False); lumped.size

16

In [5]:
goal_in_lump = lumped._subspace.find_in(goal.linear_part_as_vec()).to_numpy(dtype=cdouble)

In [6]:
obs_in_lump = [lumped._subspace.find_in(poly.linear_part_as_vec()).to_numpy(dtype=cdouble) for poly in obs]

In [7]:
obs_in_lump

[array([ 1.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        -3.03858168e-64+0.00000000e+00j,  0.00000000e+00+1.87267054e-96j,
        -2.41911047e-49-1.15734198e-64j,  1.56539742e-33+7.66823578e-49j,
         1.15224197e-33+5.64435777e-49j, -1.65825062e-33-1.22512105e-48j,
        -3.84870514e-35-3.20185155e-49j,  1.83629233e-33+1.36380148e-48j,
         1.32241503e-33+6.81633102e-49j,  2.03020526e-33+1.50290241e-48j,
         3.27535420e-33+2.85163891e-48j,  4.94943914e-33+4.70626443e-48j])]

In [17]:
compare(obs_in_lump, matmul(lumped.construct_matrices("polynomial")[0].to_numpy(dtype=cdouble), goal_in_lump))

True

In [8]:
matmul(system.construct_matrices("polynomial")[0].to_numpy(dtype=cdouble).transpose(), goal.linear_part_as_vec().to_numpy(dtype=cdouble))

array([0.70710678+0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
       0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
       0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
       0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
       0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
       0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
       0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
       0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
       0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
       0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
       0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
       0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
       0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
       0.        +0.j, 0.        +0.j, 0.        +0.j, 0.        +0.j,
      